In [ ]:
from gaussian_curve_fitting import gaussian_curve_fitting
from scipy.optimize import curve_fit
import tkinter.filedialog as tkf
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Cambria'

In [ ]:
spectrum_adr = []

In [ ]:
spectrum_adr.extend(tkf.askopenfilenames())
print(*spectrum_adr, sep="\n")

In [ ]:
deconv = gaussian_curve_fitting(spectrum_adr=spectrum_adr, option="spectrum", depth_profile=False, DM_file=True, calib=None)

In [ ]:
deconv = gaussian_curve_fitting(spectrum_adr=spectrum_adr, option="spectrum", depth_profile=False, DM_file=False, calib=[0.05, 10.0, 500.5])

In [ ]:
deconv = gaussian_curve_fitting(spectrum_adr=spectrum_adr, option="SI", depth_profile=True, DM_file=True, calib=None)

In [ ]:
deconv = gaussian_curve_fitting(spectrum_adr=spectrum_adr, option="SI", depth_profile=False, DM_file=True, calib=None)

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def gauss(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss_special_01(x, a1, a2, a3, c1, c2_1, c2_2, c3_1, c3_2, sigma1, sigma2, sigma3):
    return gauss(x, a1, c1, sigma1)+gauss(x, a2, c2_1, sigma2)+gauss(x, (3/4)*a2, c2_2, sigma2)+gauss(x, a3, c3_1, sigma3)+gauss(x, (3/4)*a3, c3_2, sigma3)

In [ ]:
bound_left = [0.0, 0.0, 0.0, 16.0, 22.5, 24.5, 25.5, 27.0, 0.0, 0.0, 0.0]
bound_right = [1.0, 1.0, 1.0, 17.0, 23.5, 25.5, 26.5, 29.5, 25.0, 25.0, 25.0]

In [ ]:
start_ev = 10.0
end_ev = 35.0

pad = 100

fitting_result = []
fit_area = []
for k in range(deconv.num_img):
    e_range = deconv.e_ranges[k]
    signal = deconv.si_data[k]
    step = deconv.steps[k]
    start_ind = find_nearest(e_range, start_ev)
    end_ind = find_nearest(e_range, end_ev)
    print(start_ev, start_ind)
    print(end_ev, end_ind)
    
    e_range_int = e_range[start_ind:end_ind]
    signal_int = signal[start_ind:end_ind]
    signal_int = signal_int / np.max(signal_int)

    slope = (signal_int[-1] - signal_int[0]) / (e_range_int[-1] - e_range_int[0])
    intercept = signal_int[0] - slope*e_range_int[0]
    bg_line = slope*e_range_int+intercept
    #bg_line = np.full(len(signal_int), np.min(signal_int))
    
    signal_int_bg_removed = signal_int - bg_line
    signal_int_bg_removed = np.append(np.zeros(pad), signal_int_bg_removed)
    signal_int_bg_removed = np.append(signal_int_bg_removed, np.zeros(pad))
    signal_int_bg_removed = signal_int_bg_removed / np.max(signal_int_bg_removed)
    e_range_int_bg_removed = np.arange(start_ev-pad*step, end_ev+pad*step, step)

    #bg_line_bg_removed = np.full(len(signal_int_bg_removed), np.min(signal_int_bg_removed))
    bg_line_bg_removed = slope*e_range_int_bg_removed+intercept / np.max(signal_int_bg_removed)
    
    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    ax.plot(e_range_int, signal_int, 'k-')
    ax.plot(e_range_int, bg_line, 'r-')
    ax.plot(e_range_int_bg_removed, signal_int_bg_removed, 'b-')
    ax.grid()
    fig.tight_layout()
    plt.show()

    popt, pcov = curve_fit(gauss_special_01, e_range_int_bg_removed, signal_int_bg_removed, 
                           bounds=(bound_left, bound_right))
    print(popt)
    fitting_result.append(popt)

    fit1 = gauss(e_range_int_bg_removed, popt[0], popt[3], popt[8])
    fit2 = gauss(e_range_int_bg_removed, popt[1], popt[4], popt[9])
    fit3 = gauss(e_range_int_bg_removed, (3/4)*popt[1], popt[5], popt[9])
    fit4 = gauss(e_range_int_bg_removed, popt[2], popt[6], popt[10])
    fit5 = gauss(e_range_int_bg_removed, (3/4)*popt[2], popt[7], popt[10])
    total = fit1 + fit2 + fit3 + fit4 + fit5
    
    fit1_area = np.trapz(fit1, e_range_int_bg_removed)
    fit2_area = np.trapz(fit2, e_range_int_bg_removed)
    fit3_area = np.trapz(fit3, e_range_int_bg_removed)
    fit4_area = np.trapz(fit4, e_range_int_bg_removed)
    fit5_area = np.trapz(fit5, e_range_int_bg_removed)
    
    fit_area.append([fit1_area, fit2_area, fit3_area, fit4_area, fit5_area])

    fig, ax = plt.subplots(1, 2, figsize=(15, 7))
    ax[0].plot(e_range_int_bg_removed, signal_int_bg_removed, 'k-')
    ax[0].fill(e_range_int_bg_removed, fit1, 'k', alpha=0.2)
    ax[0].axvline(x = popt[3], color = 'k', linestyle=":")
    ax[0].fill(e_range_int_bg_removed, fit2, 'r', alpha=0.2)
    ax[0].axvline(x = popt[4], color = 'r', linestyle=":")
    ax[0].fill(e_range_int_bg_removed, fit3, 'g', alpha=0.2)
    ax[0].axvline(x = popt[5], color = 'g', linestyle=":")
    ax[0].fill(e_range_int_bg_removed, fit4, 'b', alpha=0.2)
    ax[0].axvline(x = popt[6], color = 'b', linestyle=":")
    ax[0].fill(e_range_int_bg_removed, fit5, 'y', alpha=0.2)
    ax[0].axvline(x = popt[7], color = 'y', linestyle=":")
    ax[0].plot(e_range_int_bg_removed, total, 'r*', alpha=0.2)

    ax[1].plot(e_range_int_bg_removed, signal_int_bg_removed+bg_line_bg_removed, 'k-')
    ax[1].fill(e_range_int_bg_removed, fit1+bg_line_bg_removed, 'k', alpha=0.2)
    ax[1].axvline(x = popt[3], color = 'k', linestyle=":")
    ax[1].fill(e_range_int_bg_removed, fit2+bg_line_bg_removed, 'r', alpha=0.2)
    ax[1].axvline(x = popt[4], color = 'r', linestyle=":")
    ax[1].fill(e_range_int_bg_removed, fit3+bg_line_bg_removed, 'g', alpha=0.2)
    ax[1].axvline(x = popt[5], color = 'g', linestyle=":")
    ax[1].fill(e_range_int_bg_removed, fit4+bg_line_bg_removed, 'b', alpha=0.2)
    ax[1].axvline(x = popt[6], color = 'b', linestyle=":")
    ax[1].fill(e_range_int_bg_removed, fit5+bg_line_bg_removed, 'y', alpha=0.2)
    ax[1].axvline(x = popt[7], color = 'y', linestyle=":")
    ax[1].plot(e_range_int_bg_removed, total+bg_line_bg_removed, 'r*', alpha=0.2)

    fig.tight_layout()
    plt.show()
    
for i in range(deconv.num_img):
    print(*fitting_result[i], sep=" ")
    
for i in range(deconv.num_img):
    print(*fit_area[i], sep=" ")